In [2]:
import alborex_functions
import alborexdata
import netCDF4
import glob
import os
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap 
import numpy as np
import datetime
from importlib import reload
%matplotlib inline

In [3]:
reload(alborex_functions)

<module 'alborex_functions' from '/home/ctroupin/PycharmProjects/201705_StaresoData/alborex_functions.py'>

In [4]:
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

# Configuration

Creation of figures

In [5]:
makeFigs, makeFigsLeaflet = 1, 1

Create a logger

In [7]:
logger = alborexdata.configure_logging()

Domain

In [29]:
#coordinates = [-6.75, 3.001, 34.75, 40.]  # AlborEx
#coordinates = [-5.5, 1.3, 34.9, 38.9]      # ALMO
coordinates = [3., 11., 38., 44.]      # Stareso

Create a list of netCDF files containing SST

In [19]:
figdir = "./figures/"
figdir2 = "./figures4leaflet/"
sstdir = "/home/ctroupin/Presentations/2017/201705_StageCalvi/Data/"

In [11]:
sstfilelist = sorted(glob.glob(os.path.join(sstdir, '*SST*.nc')))
nfiles = len(sstfilelist)
logger.info("Working on {0} SST files".format(nfiles))

2017-05-08 18:44:52,880 - alborex_logger - INFO - Working on 2 SST files
2017-05-08 18:44:52,880 - alborex_logger - INFO - Working on 2 SST files


In [12]:
cmap = plt.cm.RdYlBu_r

# Data reading and plotting

Make figure directory if necessary.

In [21]:
if os.path.isdir(figdir):
    logger.debug("Figure directory {0} already exists".format(figdir))
else:
    os.makedirs(figdir)
    logger.debug("Creating figure directory {0}".format(figdir))
    
if os.path.isdir(figdir2):
    logger.debug("Figure directory {0} already exists".format(figdir2))
else:
    os.makedirs(figdir2)
    logger.debug("Creating figure directory {0}".format(figdir2))

2017-05-08 18:47:31,023 - alborex_logger - DEBUG - Figure directory ./figures/ already exists
2017-05-08 18:47:31,023 - alborex_logger - DEBUG - Figure directory ./figures/ already exists
2017-05-08 18:47:31,025 - alborex_logger - DEBUG - Figure directory ./figures4leaflet/ already exists
2017-05-08 18:47:31,025 - alborex_logger - DEBUG - Figure directory ./figures4leaflet/ already exists


## Coastline

In [24]:
reload(alborexdata)

<module 'alborexdata' from '/home/ctroupin/Publis/201703_AlborexData/python/alborexdata.py'>

In [17]:
coastdir = '../data/'
coastfile = os.path.join(coastdir, 'coastline_cartex_f3.txt')

if os.path.exists(coastfile):
    logger.info("{0} exists".format(coastfile))
    loncoast, latcoast = alborexdata.read_lonlat_coast(coastfile)
else:
    logger.warning("{0} does't exist".format(coastfile))  
    logger.warning("Return Nan values")
    loncoast, latcoast = np.nan, np.nan

2017-05-08 18:46:13,391 - alborex_logger - WARNING - ../data/coastline_cartex_f3.txt does't exist
2017-05-08 18:46:13,391 - alborex_logger - WARNING - ../data/coastline_cartex_f3.txt does't exist
2017-05-08 18:46:13,393 - alborex_logger - WARNING - Return Nan values
2017-05-08 18:46:13,393 - alborex_logger - WARNING - Return Nan values


Create the projections (only once):
* one for the normal figures (with good resolution)
* one for Leaflet (low resolution is enough).

In [33]:
m = Basemap(projection='merc', llcrnrlon=coordinates[0], llcrnrlat=coordinates[2],
                urcrnrlon=coordinates[1], urcrnrlat=coordinates[3],
                lat_ts=0.5 * (coordinates[2] + coordinates[3]), resolution='f')
# Figures without border and axes
m2 = Basemap(llcrnrlon=coordinates[0],
             llcrnrlat=coordinates[2],
             urcrnrlon=coordinates[1],
             urcrnrlat=coordinates[3], resolution = 'l', epsg=3857)

## Loop on the files

In [34]:
for sstfiles in sstfilelist:
    logger.info("Working on file: {0}".format(os.path.basename(sstfiles)))
    
    # Read data from file
    lon, lat, sst, sstqual, year, day, platform = alborex_functions.load_sst_l2(sstfiles)
    
    # Create date from year and day
    titledate = (datetime.datetime(int(year), 1, 1) + datetime.timedelta(int(day) - 1)).strftime('%Y-%m-%d')
    title = ' '.join(('MODIS', platform, ' ', titledate))
    # logger.debug(title)
    
    # Make the SST when flag > 1
    sst = np.ma.masked_where(sstqual > 1, sst)

    # Create figure name (remove .nc extension and substitute . by _)
    figname = ''.join(('_'.join(os.path.basename(sstfiles).split('.')[:-1]), '.png'))
    logger.info("Making figure {0}".format(figname))
    
    if makeFigs:
        # Normal figures
        fig = plt.figure()
        ax = plt.subplot(111)
        m.ax = ax
        pcm = m.pcolormesh(lon, lat, sst, latlon=True, vmin=15., vmax=20, cmap=cmap)

        # Add the contours if exist
        if not(np.isnan(loncoast)):
            for i in range(0, len(loncoast)):
                m.plot(np.array(loncoast[i]), np.array(latcoast[i]), color='k', linewidth=.1, latlon=True)

        alborexdata.add_map_grid(m, coordinates, dlon=1., dlat=1.,
                                 fontsize=14, 
                                 linewidth=0.2, zorder=1, color=".6")

        #m.fillcontinents(color='w', zorder=2)
        #ax.set_xlim(coordinates[0], coordinates[1])
        #ax.set_ylim(coordinates[2], coordinates[3])
        m.drawcoastlines(linewidth=0.1)
        cb = plt.colorbar(pcm, extend='both', shrink=0.8)
        cb.set_label('$^{\circ}$C', rotation=0, ha='left')
        plt.title(title)
        plt.savefig(os.path.join(figdir, figname))
        plt.close()
        
    if makeFigsLeaflet:
    
        llon, llat = m2(lon, lat)
        fig = plt.figure(frameon=False)
        ax = fig.add_axes([0, 0, 1, 1])
        m2.pcolormesh(llon, llat, sst, vmin=14., vmax=20., cmap=cmap)
        ax.axis('off')
        #ax.set_xlim(lon.min(), lon.max())
        #ax.set_ylim(lat.min(), lat.max())
        f1 = plt.gca()
        f1.axes.get_xaxis().set_ticks([])
        f1.axes.get_yaxis().set_ticks([])
        plt.savefig(os.path.join(figdir2, figname), transparent=True, 
                    bbox_inches='tight', pad_inches=0)
        plt.close()

2017-05-08 19:04:39,141 - alborex_logger - INFO - Working on file: A2017127121500.L2_LAC_SST.nc
2017-05-08 19:04:39,141 - alborex_logger - INFO - Working on file: A2017127121500.L2_LAC_SST.nc
2017-05-08 19:04:39,673 - alborex_logger - INFO - Making figure A2017127121500_L2_LAC_SST.png
2017-05-08 19:04:39,673 - alborex_logger - INFO - Making figure A2017127121500_L2_LAC_SST.png
2017-05-08 19:04:46,628 - alborex_logger - INFO - Working on file: A2017128125500.L2_LAC_SST.nc
2017-05-08 19:04:46,628 - alborex_logger - INFO - Working on file: A2017128125500.L2_LAC_SST.nc
2017-05-08 19:04:47,113 - alborex_logger - INFO - Making figure A2017128125500_L2_LAC_SST.png
2017-05-08 19:04:47,113 - alborex_logger - INFO - Making figure A2017128125500_L2_LAC_SST.png
